In [9]:
using DynamicPolynomials
using TSSOS
using MAT
using LinearAlgebra
using Printf

In [10]:
@polyvar temp1[1:4, 1:4]
@polyvar temp2[1:4, 1:4]
temp_p = temp2 * 1.1 + temp1 * 1.1

function Joint2SE3(c, s, x, y, z)
    ## we always rotate about the z axis
    T = temp_p * 0
    T[1,1] = c
    T[2,2] = c
    T[1,2] = -s
    T[2,1] = s
    T[3,3] = 1
    
    T[1,4] = c * x - s * y
    T[2,4] = s * x + c * y
    T[3,4] = z
    T[4,4] = 1
    return T
end

function SE3_Euler_num(y, p, r, lx, ly, lz)
    T = temp_p * 0
    T[1:3, 1:3] = rotz_num(y) * roty_num(p) * rotx_num(r)
    T[4,4] = 1.0
    T[1,4] = lx
    T[2,4] = ly
    T[3,4] = lz
    return T
end

function rotx_num(t)
    R = temp_p[1:3, 1:3] * 0
    R[2,2] = cos(t)
    R[3,3] = cos(t)
    R[2,3] = -sin(t)
    R[3,2] = sin(t)
    R[1,1] = 1.0
    return R
end

function roty_num(t)
    R = temp_p[1:3, 1:3] * 0
    R[1,1] = cos(t)
    R[3,3] = cos(t)
    R[1,3] = sin(t)
    R[3,1] = -sin(t)
    R[2,2] = 1.0
    return R
end

function rotz_num(t)
    R = temp_p[1:3, 1:3] * 0
    R[1,1] = cos(t)
    R[2,2] = cos(t)
    R[1,2] = -sin(t)
    R[2,1] = sin(t)
    R[3,3] = 1.0
    return R
end

function quat2rot_num(w, x, y, z)
    R = temp_p[1:3, 1:3] * 0
    nq = sqrt(w^2 + x^2 + y^2 + z^2)
    w = w / nq
    x = x / nq
    y = y / nq
    z = z / nq
    
    R[1,1] = 1 - 2 * (y^2 + z^2)
    R[2,2] = 1 - 2 * (x^2 + z^2)
    R[3,3] = 1 - 2 * (x^2 + y^2)
    
    R[1,2] = 2 * (x * y - w * z)
    R[1,3] = 2 * (w * y + x * z)
    R[2,3] = 2 * (y * z - w * x)
    
    R[2,1] = 2 * (x * y + w * z)
    R[3,1] = 2 * (- w * y + x * z)
    R[3,2] = 2 * (y * z + w * x)
    return R
end

function cross_(x, y) 
    z = temp_p[1:3, 1]
    z[1] = -x[3] * y[2] + x[2] * y[3]
    z[2] =  x[3] * y[1] - x[1] * y[3]
    z[3] = -x[2] * y[1] + x[1] * y[2]
    return z
end

function add_SO3_cons(R, eq)
    T = R' * R
#     print(T[1,1] - 1.0)
    append!(eq, [T[1,1] - 1.0])
    append!(eq, [T[2,2] - 1.0])
    append!(eq, [T[3,3] - 1.0])
    append!(eq, [T[1,2]])
    append!(eq, [T[1,3]])
    append!(eq, [T[2,3]])
    
#     append!(eq, cross_(R[1,:], R[2, :]) - R[3,:])
#     append!(eq, cross_(R[2,:], R[3, :]) - R[1,:])
#     append!(eq, cross_(R[3,:], R[1, :]) - R[2,:])

    return eq
end

function add_SE3_tran_cons(T1, T2, eq)
    T = T1 - T2
    
    append!(eq, T[1, 1])
    append!(eq, T[1, 2])
    append!(eq, T[1, 3])
    
    
    append!(eq, T[2, 1])
    append!(eq, T[2, 2])
    append!(eq, T[2, 3])
    
    
    append!(eq, T[3, 1])
    append!(eq, T[3, 2])
    append!(eq, T[3, 3])
    
    append!(eq, T[1, 4])
    append!(eq, T[2, 4])
    append!(eq, T[3, 4])    
    return eq
end

function add_SO3_tran_cons(T1, T2, eq)
    T = T1 - T2
    
#     print("eq_init = ")
#     print(size(eq))
#     print("\n")
    
#     print("trans 0 = ")
#     print(size(eq)) 
#     print("\n")
    
    append!(eq, [T[1, 1]])
    append!(eq, [T[1, 2]])
    append!(eq, [T[1, 3]])
#     print("trans 1 = ")
#     print(size(eq)) 
#     print("\n")
    
    
    append!(eq, [T[2, 1]])
    append!(eq, [T[2, 2]])
    append!(eq, [T[2, 3]])
#     print("trans 2 = ")
#     print(size(eq)) 
#     print("\n")
    
    
    append!(eq, [T[3, 1]])
    append!(eq, [T[3, 2]])
    append!(eq, [T[3, 3]])
#     print("trans 3 = ")
#     print(size(eq)) 
#     print("\n")
    
    return eq
end

function make_SE3(R, x, y, z)
    T = temp_p * 0
    T[1:3, 1:3] = R
    T[1,4] = x
    T[2,4] = y
    T[3,4] = z
    T[4,4] = 1.0
    return T
end

function SE3_inverse(T)
    Tnew = T
    Tnew[1:3, 1:3] = T[1:3, 1:3]'
    Tnew[1:3, 4] = - T[1:3, 1:3]' * T[1:3, 4]
    return Tnew
end

function SO3_skew(T)
    
    
end

# Joint2SE3(c[1], s[1], x[1], y[1], z[1])
# R = quat2rot_num(randn(), randn(), randn(), randn())
# R * R'
# rotz_num(c[1,1])
# T = SE3_Euler_num(randn(), randn(), randn(), randn(), randn(), randn())
# cross(R[1, 1,:], R[1, 2, :])

SO3_skew (generic function with 1 method)

In [11]:
function test_SO3(dof, dt, obs_flag, quat_init, mode, obs_id)

    @polyvar R[1:dof+1, 1:3, 1:3] 
    @polyvar F[1:dof+1, 1:3, 1:3]

#     @polyvar p[1:dof+1, 1:3] 
#     @polyvar v[1:dof+1, 1:3]

    @polyvar tau[1:dof, 1:3]
#     @polyvar f[1:dof, 1]

    Jb = Matrix(1.0I, 3, 3)
    Jb[1,1] = 1.0 / 10
    Jb[2,2] = 2.0 / 10
    Jb[3,3] = 1.0 / 10

    m = 1 / 2

    var = append!(reshape(R, 9 * (dof+1)), reshape(F, 9 * (dof + 1)), reshape(tau, 3 * dof))

    eq = [temp_p[1,1] * 0]

    # R_goal = quat2rot_num(1, 0, 0, 0); 
    # R_goal = quat2rot_num(1, 0, 1, 0); 
    # R_init = quat2rot_num(0.86, 0, 0.5, 0); 
    # R_goal = quat2rot_num(0.5, 0, 0.86, 0); 

    R_init = quat2rot_num(quat_init[1], quat_init[2], quat_init[3], quat_init[4]); 

    eq[1] = R[1, 1, 1] - R_init[1, 1]
    append!(eq, [R[1, 1, 2] - R_init[1, 2]])
    append!(eq, [R[1, 1, 3] - R_init[1, 3]])
    append!(eq, [R[1, 2, 1] - R_init[2, 1]])
    append!(eq, [R[1, 2, 2] - R_init[2, 2]])
    append!(eq, [R[1, 2, 3] - R_init[2, 3]])
    append!(eq, [R[1, 3, 1] - R_init[3, 1]])
    append!(eq, [R[1, 3, 2] - R_init[3, 2]])
    append!(eq, [R[1, 3, 3] - R_init[3, 3]])

    append!(eq, [F[1, 1, 1] - 1.0])
    append!(eq, [F[1, 1, 2] - 0.0])
    append!(eq, [F[1, 1, 3] - 0.0])
    append!(eq, [F[1, 2, 1] - 0.0])
    append!(eq, [F[1, 2, 2] - 1.0])
    append!(eq, [F[1, 2, 3] - 0.0])
    append!(eq, [F[1, 3, 1] - 0.0])
    append!(eq, [F[1, 3, 2] - 0.0])
    append!(eq, [F[1, 3, 3] - 1.0])

    print(size(eq))
    # print("hahahah\n")
    f_act = 0

    g = [0;0;-9.81]

    ineq = []

    Qc = [1e-1; 10; 1e-1; 1] # R w p v
    Rc = 1e-2 # rt rf
    Pc = [100; 10; 100; 100] * 1.1 / 1.1 # 

    for k = 1:dof
    #     print("hahahah\n")
        ## kinematics constraints
#         eq = add_SO3_cons(R[k+1,:,:], eq) # SO3 constraints
#         eq = add_SO3_cons(F[k+1,:,:], eq) # SO3 constraints
        eq = add_SO3_tran_cons(R[k+1, :, :], R[k, :, :]' * F[k,: , :], eq)

        ## dynamics constraints
        Fkp1 = F[k+1, :, :];
        Fk =   F[k, :, :];
        Mkp1 = Fkp1 * Jb - Jb * Fkp1'
        Mk = Jb * Fk - Fk' * Jb
        dM = Mkp1 - Mk # SO3

        append!(eq, [dM[1, 2] + (-tau[k, 3]) * dt^2 ])
        append!(eq, [dM[1, 3] + (tau[k, 2]) * dt^2 ])
        append!(eq, [dM[2, 3] + (-tau[k, 1]) * dt^2 ])


        tau_max = 5
    #     f_max = 7


        R_diff = R[k, :, :] - Matrix(1.0I, 3, 3) # + randn(3, 3) * 10
        R_diff = R_diff' * R_diff


        F_cost = F[k, : , :] - Matrix(1.0I, 3, 3) # + randn(3, 3) * 10
        F_cost = F_cost' * F_cost

        R_cost = R_diff[1,1] + R_diff[2,2] + R_diff[3,3]
        w_cost = F_cost[1, 1] + F_cost[2, 2] + F_cost[3, 3]

        f_act = f_act + Rc * (tau[k, 1]^2 + tau[k, 2]^2 + tau[k, 3]^2) # + Rc * f[k, 1]^2
        f_act = f_act + Qc[1] * R_cost
        f_act = f_act + Qc[2] * w_cost
        
        ineq = append!(ineq, [10.0 - sum(F[k+1, :, :].^2)])
        ineq = append!(ineq, [10.0 - sum(R[k+1, :, :].^2)])
    end

    # orientation error
    p_goal = [0, 0,0.0]

    R_diff = R[end, :, :] - Matrix(1.0I, 3, 3) # + randn(3, 3) * 10
    R_diff = R_diff' * R_diff
    f_R = R_diff[1,1] + R_diff[2,2] + R_diff[3,3]

    # angular velocity error
    F_diff = F[end, :, :] - Matrix(1.0I, 3, 3) # + randn(3, 3) * 10
    F_diff = F_diff' * F_diff
    f_w = F_diff[1,1] + F_diff[2,2] + F_diff[3,3]

    # position cost
#     f_p = sum((p[end,:] - p_goal).^2)
#     f_v = sum((v[end,:]).^2)

    cost = f_act * dt / 0.25
    cost = cost + Pc[1] * f_R + Pc[2] * f_w # + Pc[3] * f_p + Pc[4] * f_v

#     cost = cost + 1e-2 * sum(var.^4)
    
    
    
    pop = append!([cost * 10], ineq, eq)
    eq_len = length(eq)

    order = 2
    
    time_start = time()
    if mode == 1
        opt,sol,data,model,sol_approx =cs_tssos_first_export_tsl(pop, var, order, numeq=eq_len, CS="NC", TS = "MD", solution=true, QUIET=false, MomentOne=false)
    elseif mode == 2
        opt,sol,data,model,sol_approx =cs_tssos_first_export_tsl(pop, var, order, numeq=eq_len, CS="NC", TS = "block", solution=true, QUIET=false, MomentOne=false)
    elseif mode == 3
        opt,sol,data,model,sol_approx =cs_tssos_first_export_tsl(pop, var, order, numeq=eq_len, CS="NC", TS = false, solution=true, QUIET=false, MomentOne=false)
    end
    time_end = time()
    elapsed = time_end - time_start

    
    moment = []
    for k = 1:length(data.Mmatrix)
        append!(moment, [convert(Matrix{Float64}, data.Mmatrix[k])]) # data.Mmatrix[k])
    end


    log = Dict("sol" => sol, "sol_approx" => sol_approx, "dof" => dof, 
        "moment" => moment, "opt" => opt, "dt"=>dt, "Mass"=>m, "Inertial"=>Jb, 
        "quat_init"=> quat_init, "Qc"=> Qc, "Rc"=>Rc, "Pc"=>Pc, "obs_flag"=>obs_flag,
        "solver_flag"=>data.flag, "elapsed"=>elapsed)
    return log
end
quat_list = [[0.86;0;0.5;0], [1;0;0;0], [1;0;1;0], [0.5, 0, 0.85, 0], [0,0,1,0]]
obs_list = [false; true]
mode_list = [1; 2; 3] # step using pure cs at this moment. 

dof = 2
dt = 0.1
h = 1
k = 1

log = test_SO3(dof, dt, obs_list[h], quat_list[k], 3, 2)

(18,)

LoadError: UndefVarError: cs_tssos_first_export_tsl not defined

In [12]:
function test_drone(dof, dt, obs_flag, quat_init, mode, obs_id)

    @polyvar R[1:dof+1, 1:3, 1:3] 
    @polyvar F[1:dof+1, 1:3, 1:3]

    @polyvar p[1:dof+1, 1:3] 
    @polyvar v[1:dof+1, 1:3]

    @polyvar tau[1:dof, 1:3]
    @polyvar f[1:dof, 1]

    Jb = Matrix(1.0I, 3, 3)
    Jb[1,1] = 1.0 / 10
    Jb[2,2] = 2.0 / 10
    Jb[3,3] = 1.0 / 10

    m = 1 / 2

    var = append!(reshape(R, 9 * (dof+1)), reshape(F, 9 * (dof + 1)), reshape(p, 3 * (dof + 1)), reshape(v, 3 * (dof + 1)), reshape(tau, 3 * dof), reshape(f, dof))

    eq = [temp_p[1,1] * 0]

    # R_goal = quat2rot_num(1, 0, 0, 0); 
    # R_goal = quat2rot_num(1, 0, 1, 0); 
    # R_init = quat2rot_num(0.86, 0, 0.5, 0); 
    # R_goal = quat2rot_num(0.5, 0, 0.86, 0); 

    R_init = quat2rot_num(quat_init[1], quat_init[2], quat_init[3], quat_init[4]); 

    eq[1] = R[1, 1, 1] - R_init[1, 1]
    append!(eq, [R[1, 1, 2] - R_init[1, 2]])
    append!(eq, [R[1, 1, 3] - R_init[1, 3]])
    append!(eq, [R[1, 2, 1] - R_init[2, 1]])
    append!(eq, [R[1, 2, 2] - R_init[2, 2]])
    append!(eq, [R[1, 2, 3] - R_init[2, 3]])
    append!(eq, [R[1, 3, 1] - R_init[3, 1]])
    append!(eq, [R[1, 3, 2] - R_init[3, 2]])
    append!(eq, [R[1, 3, 3] - R_init[3, 3]])

    append!(eq, [F[1, 1, 1] - 1.0])
    append!(eq, [F[1, 1, 2] - 0.0])
    append!(eq, [F[1, 1, 3] - 0.0])
    append!(eq, [F[1, 2, 1] - 0.0])
    append!(eq, [F[1, 2, 2] - 1.0])
    append!(eq, [F[1, 2, 3] - 0.0])
    append!(eq, [F[1, 3, 1] - 0.0])
    append!(eq, [F[1, 3, 2] - 0.0])
    append!(eq, [F[1, 3, 3] - 1.0])

    append!(eq, [p[1,1] - 1.0])
    append!(eq, [p[1,2] - 1.0])
    append!(eq, [p[1,3] - 3.0])

    append!(eq, [v[1,1] - 0.])
    append!(eq, [v[1,2] - 0.])
    append!(eq, [v[1,3] - 0.])
    # append!(eq, [v[1,1] - randn() * 0.1])
    # append!(eq, [v[1,2] - randn() * 0.1])
    # append!(eq, [v[1,3] - randn() * 0.1])

    print(size(eq))
    # print("hahahah\n")
    f_act = 0

    g = [0;0;-9.81]

    ineq = []

    Qc = [1e-1; 10; 1e-1; 1] # R w p v
    Rc = 1e-2 # rt rf
    Pc = [100; 10; 100; 100] * 1.1 / 1.1 # 

    for k = 1:dof
    #     print("hahahah\n")
        ## kinematics constraints
        eq = add_SO3_cons(R[k+1,:,:], eq) # SO3 constraints
        eq = add_SO3_cons(F[k+1,:,:], eq) # SO3 constraints
        eq = add_SO3_tran_cons(R[k+1, :, :], R[k, :, :] * F[k,: , :], eq)

        p_diff = p[k+1, :] - (p[k, :] + dt * R[k,:, :] * v[k,:]) # position

#         append!(eq, [p_diff[1]])
#         append!(eq, [p_diff[2]])
#         append!(eq, [p_diff[3]])
    #     print("hahahah\n")

        ## dynamics constraints
        Fkp1 = F[k+1, :, :];
        Fk =   F[k, :, :];
        Mkp1 = Fkp1 * Jb - Jb * Fkp1'
        Mk = Jb * Fk - Fk' * Jb
        dM = Mkp1 - Mk # SO3

        append!(eq, [dM[1, 2] + (-tau[k, 3]) * dt^2 ])
        append!(eq, [dM[1, 3] + (tau[k, 2]) * dt^2 ])
        append!(eq, [dM[2, 3] + (-tau[k, 1]) * dt^2 ])


        tau_max = 5
    #     f_max = 7

    #     append!(ineq, [f_max - f[k]])
    #     append!(ineq, [f_max + f[k]])
        append!(ineq, [tau_max - tau[k, 1]])
        append!(ineq, [tau_max + tau[k, 1]])
        append!(ineq, [tau_max - tau[k, 2]])
        append!(ineq, [tau_max + tau[k, 2]])
        append!(ineq, [tau_max - tau[k, 3]])
        append!(ineq, [tau_max + tau[k, 3]])

        append!(ineq, [p[k+1, 3]])

        if obs_flag == true
            if obs_id == 1
                append!(ineq, [(p[k+1, 1])^2 + (p[k+1, 2] - 0.5)^2 - 0.25])
            else
                append!(ineq, [(p[k+1, 1] - 0.6)^2 + (p[k+1, 2] - 0.5)^2 - 0.25])
            end
#             append!(ineq, [(p[k+1, 1] - 0.5)^2 + (p[k+1, 2] - 0.5)^2 - 0.25])
        end
    #     append!(ineq, [p[k+1, 3]^2 - (p[k+1, 2]^2 + p[k+1, 3]^2) / 6])
    #     append!(ineq, [(p[k+1, 1] - 0.0)^2 + (p[k+1, 2] - 0.5)^2 - 0.25])

        v_diff = m * v[k+1,:] - (m * Fk' * v[k, :] + ([0; 0; 1.0]* f[k]+ m * R[k+1,:,:]' * g) * dt )

#         append!(eq, [v_diff[1]])
#         append!(eq, [v_diff[2]])
#         append!(eq, [v_diff[3]])
    #     print("hahahah\n")

        R_diff = R[k, :, :] - Matrix(1.0I, 3, 3)
        R_diff = R_diff' * R_diff


        F_cost = F[k, : , :] - Matrix(1.0I, 3, 3)
        F_cost = F_cost' * F_cost

        R_cost = R_diff[1,1] + R_diff[2,2] + R_diff[3,3]
        w_cost = F_cost[1, 1] + F_cost[2, 2] + F_cost[3, 3]
        p_cost = sum(p[k, :].^2)
        v_cost = sum(v[k, :].^2)

        f_act = f_act + Rc * (tau[k, 1]^2 + tau[k, 2]^2 + tau[k, 3]^2) + Rc * f[k, 1]^2
        f_act = f_act + Qc[1] * R_cost
        f_act = f_act + Qc[2] * w_cost
        f_act = f_act + Qc[3] * p_cost
        f_act = f_act + Qc[4] * v_cost
    end

    # orientation error
    p_goal = [0, 0,0.0]

    R_diff = R[end, :, :] - Matrix(1.0I, 3, 3)
    R_diff = R_diff' * R_diff
    f_R = R_diff[1,1] + R_diff[2,2] + R_diff[3,3]

    # angular velocity error
    F_diff = F[end, :, :] - Matrix(1.0I, 3, 3)
    F_diff = F_diff' * F_diff
    f_w = F_diff[1,1] + F_diff[2,2] + F_diff[3,3]

    # position cost
    f_p = sum((p[end,:] - p_goal).^2)
    f_v = sum((v[end,:]).^2)

    cost = f_act * dt / 0.25
    cost = cost + Pc[1] * f_R + Pc[2] * f_w + Pc[3] * f_p + Pc[4] * f_v

#     cost = cost + 1e-2 * sum(var.^4)
    
    pop = append!([cost * 10], ineq, eq)
    eq_len = length(eq)

    order = 2
    
    time_start = time()
    if mode == 1
        opt,sol,data =cs_tssos_first(pop, var, order, numeq=eq_len, CS="NC", TS = "MD", solution=true, QUIET=false, MomentOne=false)
    elseif mode == 2
        opt,sol,data =cs_tssos_first(pop, var, order, numeq=eq_len, CS="NC", TS = "block", solution=true, QUIET=false, MomentOne=false)
    elseif mode == 3
        opt,sol,data =cs_tssos_first(pop, var, order, numeq=eq_len, CS="NC", TS = false, solution=true, QUIET=false, MomentOne=false)
    end
    time_end = time()
    elapsed = time_end - time_start

    
    moment = []
    for k = 1:length(data.moment)
        append!(moment, [convert(Matrix{Float64}, data.moment[k])]) # data.Mmatrix[k])
    end


    log = Dict("sol" => sol,"dof" => dof, #  "sol_approx" => sol_approx, 
        "moment" => moment, "opt" => opt, "dt"=>dt, "Mass"=>m, "Inertial"=>Jb, 
        "quat_init"=> quat_init, "Qc"=> Qc, "Rc"=>Rc, "Pc"=>Pc, "obs_flag"=>obs_flag,
        "solver_flag"=>data.flag, "elapsed"=>elapsed)
    return log
end

test_drone (generic function with 1 method)

In [ ]:
quat_list = [[1;0;0;0], [0.86;0;0.5;0], [1;0;1;0], [0.5, 0, 0.85, 0], [0,0,1,0]] # 60deg 0deg 90deg 120deg 180deg
obs_list = [false; true]
mode_list = [1; 2; 3]

dof = 3
dt = 0.25 * 3 / 4

for j = 3 # sparse mode
    for k = 1:5 # 1:3 # initial condition
        for h = 2 # obs or not
            log = test_drone(dof, dt, obs_list[h], quat_list[k], mode_list[j], 2) # new obs at the center
            file_name = "log_s30_obs_" * string(j) * "_" * string(k) * "_" * string(h) * ".mat"
            matwrite(file_name, log)
        end     
    end
end

(24,)*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[7, 6, 5, 9, 18, 1, 2]
[27, 30, 18, 2, 2, 21, 3]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.1092755 seconds. The maximal size of cliques is 18.
Assembling the SDP...
There are 23074 affine constraints.
SDP assembling time: 0.92847 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 23074           
  Cones                  : 0               
  Scalar variables       : 11491           
  Matrix variables       : 127             
  Integer variables      : 0               

Optimizer started.
Presolve st